In [ ]:
import csv
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F 
from torch import optim
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [ ]:
def timePointList(dataList, timePointBracket):
    tpList = []
    for i in range(0,int(np.ma.count(dataList)/timePointBracket)):
        if(np.ma.count(dataList) < i*timePointBracket):
            #This is the last iteration
            delta = (dataList[-1] - dataList[(i*timePointBracket)])/(np.ma.count(dataList) - (i*timePointBracket))
            tpList.append(delta)
        else:
            delta = (dataList[(i*timePointBracket) + timePointBracket - 1] - dataList[(i*timePointBracket)])/timePointBracket
            tpList.append(delta)
    return tpList

def generateMotionGraphs(dataX, dataY, dataZ, dataYaw, dataPitch, dataRoll, timepoints):
    deltaX = timePointList(dataX,timepoints)
    deltaY = timePointList(dataY,timepoints)
    deltaZ = timePointList(dataZ,timepoints)
    deltaYaw = timePointList(dataYaw, timepoints)
    deltaPitch = timePointList(dataPitch, timepoints)
    deltaRoll = timePointList(dataRoll, timepoints)

    plt.figure(1)
    plt.plot(deltaX, label = '△X')
    plt.plot(deltaY, label = '△Y')
    plt.plot(deltaZ, label = '△Z')
    plt.title("Body Motion Speed")
    plt.legend()
    plt.savefig("BodyMotion.jpg", dpi=200)

    plt.figure(2)
    plt.plot(deltaYaw, label = '△Yaw')
    plt.plot(deltaPitch, label = '△Pitch')
    plt.plot(deltaRoll, label = '△Roll')
    plt.title("Head Motion Speed")
    plt.legend()
    plt.savefig("HeadMotion.jpg", dpi=200)


In [ ]:
def create_dataset1(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

def generateDataset(dataArr, look_back = 125):
    dataset=dataArr.reshape(-1, 1)
    train_size = int(len(dataset) * 0.9)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

    trainX, trainY = create_dataset1(train, look_back)
    testX, testY = create_dataset1(test, look_back)
    trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
    testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
    trainX = torch.from_numpy(trainX).float()
    trainY = torch.from_numpy(trainY).float()
    testX = torch.from_numpy(testX).float()
    testY = torch.from_numpy(testY).float()
    return trainX, trainY, testX, testY

In [ ]:
## removed from code currently
def noiseFilter(dataCol):
    N  = 3    # Filter order
    Wn = 0.1 # Cutoff frequency
    B, A = signal.butter(N, Wn, output='ba')
    ts = dataCol[0:]
    smooth_data = signal.filtfilt(B,A, dataCol[0:])
    #plt.plot(ts,'r-')
    #plt.plot(smooth_data[0:],'b-')
    #plt.show()
    return smooth_data

In [ ]:
class LSTM2(nn.Module):
    def __init__(self, n_step = 125, hidden_layers=32, numLayers = 1):
        super(LSTM2, self).__init__()
        self.hidden_layers = hidden_layers
        self.numLayers = numLayers
        self.lstm1 = nn.LSTM(n_step, self.hidden_layers, batch_first=True, num_layers = numLayers)
        self.linear = nn.Linear(self.hidden_layers, 1)
        
    def forward(self, inputVal):
        hidden_state = torch.randn(self.numLayers, inputVal.size(0), 32).float()
        cell_state = torch.randn(self.numLayers, inputVal.size(0), 32).float()
        hidden = (hidden_state, cell_state)
        h_t, c_t = self.lstm1(inputVal.float(), hidden)
        outputs = self.linear(h_t) # output from the last FC layer
        return outputs

In [ ]:
def training_loop(n_epochs, model, optimiser, loss_fn, train_input, train_target):
    model.train()
    trainLoss = []
    train_target = train_target.reshape(train_target.size(0), 1, 1)
    for i in range(n_epochs):
        optimiser.zero_grad()
        out = model(train_input)
        loss = loss_fn(out, train_target)
        loss.backward()    
        optimiser.step()
        trainLoss.append(loss)
        print("Step: {}, Loss: {}".format(i, loss))
    return trainLoss

def eval_model(model, loss_fn, test_input, test_target):
        model.eval()
        test_target = test_target.reshape(test_target.size(0), 1, 1)
        with torch.no_grad():
            pred = model(test_input)
            loss = loss_fn(pred, test_target)
            return loss, pred
        

In [ ]:
def trainModel(num_epochs, model, optimiser, criterion, trainX, trainY, plotsFolderName, colName):
    model = model.float()
    train_loss = training_loop(num_epochs, model, optimiser, criterion, trainX, trainY)
    lossArr = []
    for i in range(0,len(train_loss)):
        lossArr.append(train_loss[i].item())
    plt.plot(lossArr, label = "Training Loss")
    plt.legend()
    plt.savefig(plotsFolderName+colName+"trainLoss.jpg", dpi=200)
    plt.close()

In [ ]:
testYPlot = []
predYPlot = []

In [ ]:
def evaluateModel(model, criterion, testX, testY, plotsFolderName, colName, rotFlag):
    test_loss, prediction = eval_model(model, criterion, testX, testY)
    print(test_loss.item())
    if rotFlag == 1:
        print("only fo rroo")
        prediction = prediction.detach().numpy()
        prediction = prediction.flatten()
        prediction = np.where(prediction, np.rad2deg(prediction), prediction)
        testY = np.where(testY, np.rad2deg(testY), testY)
    else:
        prediction = prediction.reshape(prediction.size(0))
    plt.figure(figsize=(8, 6), dpi=100)
    plt.plot(testY[::50] , label = "Actual")
    plt.plot(prediction[::50], label = "Predicted")
    plt.legend()
    plt.savefig(plotsFolderName+colName+"truth.jpg", dpi=200)
    plt.close()
    #return testY, prediction.reshape(prediction.size(0))
    return testY, prediction, test_loss

In [ ]:
Xloss = []
Yloss = []
Zloss = []
Yawloss = []
Rollloss = []
Pitchloss = []

for i in range(1,2): ## change the limit here to 18 for the whole dataset
    fileName = r"dataset/node" + str(i) +  "mobility.csv"
    resultsFolder = r"results/node" + str(i) + "mobility/"
    plotsFolderName = r"results/node" + str(i) + "mobility/plots/"
    df = pd.read_csv (fileName)
    tot = 0
    rotFlag = 0
    for (columnName, columnData) in df.iteritems():
        
        print("Predicting for " + columnName)
        dataCol = df[columnName].to_numpy()
        tot = tot + 1
        if(tot < 4):
            rotFlag = 0
        else:
            rotFlag = 1
            print("only for roation")
            dataCol = np.where(dataCol, np.deg2rad(dataCol), dataCol)
        
        trainDataX, trainDataY, testDataX, testDataY = generateDataset(dataCol, 125)
        model = LSTM2(n_step = 125, numLayers = 2)
        criterion = nn.MSELoss()
        optimiser = optim.Adam(model.parameters(), lr=0.01)
        trainModel(100, model, optimiser, criterion, trainDataX, trainDataY,plotsFolderName,columnName)
        torch.save(model.state_dict(), resultsFolder+columnName+"savedModel.pt")
        testYPlot, predYPlot, lossVal = evaluateModel(model, criterion, testDataX, testDataY, plotsFolderName, columnName, rotFlag)
        
        if tot == 1:
            Xloss.append(lossVal)
        if tot == 2:
            Yloss.append(lossVal)
        if tot == 3:
            Zloss.append(lossVal)
        if tot == 4:
            Yawloss.append(lossVal)
        if tot == 5:
            Pitchloss.append(lossVal)
        if tot == 6:
            Rollloss.append(lossVal)     

In [ ]:

print(np.mean(Xloss))
print(np.mean(Yloss))
print(np.mean(Zloss))
print(np.mean(Yawloss))
print(np.mean(Rollloss))
print(np.mean(Pitchloss))

In [ ]:
## CNN Section
from torch.utils.data import Dataset,DataLoader
class Feedforward(torch.nn.Module):
    def __init__(self, n_step, hidden_size):
        super(Feedforward, self).__init__()
        self.conv1d = nn.Conv1d(n_step,hidden_size,kernel_size=1)
        self.relu = nn.ReLU(inplace=True)
        self.fc1 = nn.Linear(300, 100)
        self.fc2 = nn.Linear(100, 50)
        self.outLay = nn.Linear(50,1)
    def forward(self, x):
        x = self.conv1d(x)
        x = self.relu(x)
        x = x.view(-1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.outLay(x)
        return x

def Train(model):
    
    running_loss = .0
    
    model.train()
    
    for idx, (inputs,labels) in enumerate(train_loader):
        inputs = inputs
        labels = labels
        optimizer.zero_grad()
        preds = model(inputs)
        loss = criterion(preds,labels)
        loss.backward()
        optimizer.step()
        running_loss += loss
        
    train_loss = running_loss/len(train_loader)
    print(f'train_loss {train_loss}')
    return train_loss
    
    
def Valid(model):
    running_loss = .0
    
    model.eval()
    
    with torch.no_grad():
        for idx, (inputs, labels) in enumerate(test_loader):
            inputs = inputs
            labels = labels
            optimizer.zero_grad()
            preds = model(inputs)
            loss = criterion(preds,labels)
            running_loss += loss
            
        valid_loss = running_loss/len(test_loader)
        print(f'valid_loss {valid_loss}')
        return valid_loss, preds
        
class NewDataset(Dataset):
    def __init__(self,feature,target):
        self.feature = feature
        self.target = target
    
    def __len__(self):
        return len(self.feature)
    
    def __getitem__(self,idx):
        item = self.feature[idx]
        label = self.target[idx]
        
        return item,label


for i in range(1,2): ##Change the limit here to 18 for whole dataset
    fileName = r"dataset/node" + str(i) +  "mobility.csv"
    resultsFolder = r"results/node" + str(i) + "mobility/"
    plotsFolderName = r"results/node" + str(i) + "mobility/plots/"
    df = pd.read_csv (fileName)
    tot = 0;
    for (columnName, columnData) in df.iteritems():
        tot = tot + 1
        if(tot < 4):
            continue
        print("Predicting for " + columnName)
        dataCol = df[columnName].to_numpy()
        dataCol = np.where(dataCol, np.deg2rad(dataCol), dataCol)
        trainDataX, trainDataY, testDataX, testDataY = generateDataset(dataCol, 125)
        trainDataX = trainDataX.reshape(trainDataX.shape[0],125)
        trainDataX = trainDataX.reshape(trainDataX.shape[0],trainDataX.shape[1],1)
        
        testDataX = testDataX.reshape(testDataX.shape[0],125)
        testDataX = testDataX.reshape(testDataX.shape[0],testDataX.shape[1],1)
        
        train = NewDataset(trainDataX,trainDataY)
        valid = NewDataset(testDataX,testDataY)
        print(trainDataX.shape)
        print(testDataX.shape)
        train_loader = torch.utils.data.DataLoader(train,batch_size=6,shuffle=False)
        test_loader = torch.utils.data.DataLoader(valid,batch_size=6,shuffle=False)
        model = Feedforward(125, 50)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=0.01)
        epochs = 5
        train_loss = []
        test_loss = []
        test_pred = 0
        for epoch in range(epochs):
            print('epochs {}/{}'.format(epoch+1,epochs))
            temp_train_loss = Train(model)
            train_loss.append(temp_train_loss.detach().numpy())
        
        model.eval()
        prediction = []
        batch_size = 6
        iterations =  int(testDataY.shape[0]/6)

        for i in range(iterations):
            preds = model(torch.tensor(testDataY[batch_size*i:batch_size*(i+1)]))
            prediction.append(preds.detach().numpy())
            
        plt.figure(2)
        plt.plot(testDataY[::50])
        plt.plot(prediction[::50])
        break

In [ ]:
df = pd.read_csv (r"dataset/node3mobility.csv")

dataX = df['X'].to_numpy()
dataY = df['Y'].to_numpy()
dataZ = df['Z'].to_numpy()
dataYaw = df['Yaw'].to_numpy()
dataPitch = df['Pitch'].to_numpy()
dataRoll = df['Roll'].to_numpy()

generateMotionGraphs(dataX, dataY, dataZ, dataYaw, dataPitch, dataRoll, 125)